In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:600px; }</style>")) 

# Precompute embeddings

Resourses and useful links:

 - ...

In [ ]:
#nbx

#ximp
import matplotlib.pyplot as plt
from IPython.display import Audio, display
plot = plt.plot

In [ ]:
#nbx
#ximp
import warnings
warnings.filterwarnings("ignore")
#ximp
import torch
import numpy as np
from cgnai.utils import cgnai_home, sliding_window_ind
from cgnai.fileio import load, dump, ls
from cgnai.audio.mels import cut_up
import torchaudio
import librosa
from pathlib import Path
from cgnai.audio.utils import compute_embedding

In [ ]:
#nbx
import numpy as np
from pyannote.audio import Pipeline
from pyannote.audio import Inference

seg_pipe = Pipeline.from_pretrained("pyannote/speaker-segmentation")

def compute_segs(w,sr):
    res  = seg_pipe({"waveform": w[[0]], "sample_rate": sr})
    segs = res.get_timeline().segments_list_
    return np.array([(round(s.start * sr), round(s.end*sr)) for s in segs])

va_pipe = Pipeline.from_pretrained("pyannote/voice-activity-detection")

def compute_va(w, sr):
    """Voice Activity."""
    res = va_pipe({"waveform": w[[0]], "sample_rate": sr})
    return np.array([(round(x.start*sr), round(x.end*sr)) for x in res._tracks.keys()])

In [ ]:
# def plot(Z):
#     plt.figure(figsize=(4,1))
#     for i,z in enumerate(Z):
#         if isinstance(z, slice): z = [z.start, z.stop]  
#         plt.plot([z[0]/sr,z[1]/sr],[i,i])

In [ ]:
# def accum(Z, d):
#     v = np.zeros(len(S))

#     for z in Z:
#         if isinstance(z, slice): z = [z.start, z.stop]
#         r0 = (z[0] - 25600)//d + 1
#         r1 = z[1]//d  
#         i0 = max(r0,0)
#         i1 = min(r1,len(v)-1)

#         v[i0:i1] += 1
        
#     return v

## Load data

In [ ]:
#nbx
#xargs
xdir  = Path("./")
xtask = 0 

In [ ]:
#nbx
path = cgnai_home()/"shared/podverse/data/politik_podcast"
is_mp3 = lambda s: s.endswith(".mp3")

fnames = list(filter(is_mp3, ls(path).files))
len(fnames)

In [ ]:
#nbx

#xarg
t = 1; range(19)

fname = fnames[t]
torchaudio.set_audio_backend("sox_io") # Can't load mp3 otherwise on my Macbook
wav, sr =  torchaudio.load(str(path/fname))


print(fname)
print(wav.size())

In [ ]:
#nbx
#xarg
dur  = 1.6; [0.8, 1.6, 2.4, 3.2, 4.0] # cut width for the embeddings 

step = min(dur, 1.0)

emb = Inference("pyannote/embedding", window="sliding", duration=dur, step=step)

In [ ]:
#nbx
x = emb({"waveform":wav[[0]], "sample_rate": sr})
x = x.data

dump(x, str(xdir/"emb.pkl")) ####

In [ ]:
#nbx
x = x/np.linalg.norm(x, axis=1, keepdims=True)
d = x@x.T

dump(d, xdir/"sim.pkl") ####

fig, axs = plt.subplot_mosaic([[0],[1]], 
                          sharey=False,
                          gridspec_kw = dict(width_ratios=[1], height_ratios=[4,1]),
                          figsize=(4, 6), layout="constrained")
axs[0].imshow(d);
axs[1].hist(d.flatten(), bins=np.linspace(0,1,50));

plt.savefig(xdir/"summary.jpg", bbox_inches='tight', dpi=100) ####

# NBX

In [ ]:
from cgnai.nbx import create_bundle
create_bundle("_nbx_embeddings_fixed")

# Analysis